In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [6]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
home_sales_df.createOrReplaceTempView("home_sales")
home_sales_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [44]:
from pyspark.sql.functions import expr
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
home_sales_avg_price = spark.sql("""
Select date_built as year, CONCAT('$', format_number(avg_price,2)) as avg_price
from(
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY date_built
    ORDER BY date_built
) as subquery
order by year
""")
home_sales_avg_price.show()


+----+-----------+
|year|  avg_price|
+----+-----------+
|2010|$296,800.75|
|2011|$302,141.90|
|2012|$298,233.42|
|2013|$299,999.39|
|2014|$299,073.89|
|2015|$307,908.86|
|2016|$296,050.24|
|2017|$296,576.69|
+----+-----------+



In [39]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
home_sales_avg_price_built = spark.sql("""
    SELECT date_built as year,CONCAT('$', FORMAT_NUMBER(avg_price, 2)) AS avg_price
    FROM (
        SELECT date_built, ROUND(AVG(price), 2) AS avg_price
        FROM home_sales
        WHERE bedrooms = 3 AND bathrooms = 3
        GROUP BY date_built
    ) AS subquery
    ORDER BY year
""")
home_sales_avg_price_built.show()


+----+-----------+
|year|  avg_price|
+----+-----------+
|2010|$292,859.62|
|2011|$291,117.47|
|2012|$293,683.19|
|2013|$295,962.27|
|2014|$290,852.27|
|2015|$288,770.30|
|2016|$290,555.07|
|2017|$292,676.79|
+----+-----------+



In [41]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
  Select date_built as year, concat('$', format_number(avg_price, 2)) AS avg_price
  from (
    select date_built, round(avg(price), 2) as avg_price
    from home_sales
    where bedrooms = 3
    and bathrooms = 3
    and floors = 2
    and sqft_living >= 2000
    group by date_built
  ) as subquery
  order by year
"""
spark.sql(query).show()

+----+-----------+
|year|  avg_price|
+----+-----------+
|2010|$285,010.22|
|2011|$276,553.81|
|2012|$307,539.97|
|2013|$303,676.79|
|2014|$298,264.72|
|2015|$297,609.97|
|2016|$293,965.10|
|2017|$280,317.58|
+----+-----------+



In [46]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
query = """
     Select view, date_built as year, concat('$', format_number(avg_price, 2)) AS avg_price
     from (
            select view, date_built, round(avg(price), 2) as avg_price
            from home_sales
            group by view, date_built
            having avg_price >= 350000
          ) as subquery
    order by view desc
"""
spark.sql(query).show()

start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

+----+----+-------------+
|view|year|    avg_price|
+----+----+-------------+
| 100|2013|$1,321,942.00|
| 100|2014|$1,048,368.75|
| 100|2017|  $805,038.00|
| 100|2011|  $973,403.00|
| 100|2010|$1,040,832.50|
| 100|2015|  $716,333.50|
| 100|2016|$1,154,589.25|
| 100|2012|$1,069,511.25|
|  99|2010|$1,046,322.25|
|  99|2015|  $959,850.80|
|  99|2014|  $979,975.50|
|  99|2017|$1,107,391.00|
|  99|2016|$1,105,896.50|
|  99|2012|  $992,771.50|
|  99|2013|$1,286,284.50|
|  99|2011|$1,158,352.67|
|  98|2016|$1,202,241.00|
|  98|2012|$1,121,207.00|
|  98|2013|  $998,008.40|
|  98|2011|  $991,078.00|
+----+----+-------------+
only showing top 20 rows

--- 0.00010228157043457031 seconds ---


In [47]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [48]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [49]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
query = """
     Select view, date_built as year, concat('$', format_number(avg_price, 2)) AS avg_price
     from (
            select view, date_built, round(avg(price), 2) as avg_price
            from home_sales
            group by view, date_built
            having avg_price >= 350000
          ) as subquery
    order by view desc
"""
spark.sql(query).show()

start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))


+----+----+-------------+
|view|year|    avg_price|
+----+----+-------------+
| 100|2013|$1,321,942.00|
| 100|2014|$1,048,368.75|
| 100|2017|  $805,038.00|
| 100|2011|  $973,403.00|
| 100|2010|$1,040,832.50|
| 100|2015|  $716,333.50|
| 100|2016|$1,154,589.25|
| 100|2012|$1,069,511.25|
|  99|2010|$1,046,322.25|
|  99|2015|  $959,850.80|
|  99|2014|  $979,975.50|
|  99|2017|$1,107,391.00|
|  99|2016|$1,105,896.50|
|  99|2012|  $992,771.50|
|  99|2013|$1,286,284.50|
|  99|2011|$1,158,352.67|
|  98|2016|$1,202,241.00|
|  98|2012|$1,121,207.00|
|  98|2013|  $998,008.40|
|  98|2011|  $991,078.00|
+----+----+-------------+
only showing top 20 rows

--- 9.1552734375e-05 seconds ---


In [50]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
partitioned_data = home_sales_df.write.partitionBy("date_built").parquet("home_sales_partitioned.parquet")

In [57]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_partitioned.parquet")

In [72]:
# 12. Create a temporary table for the parquet data.
temp_df = parquet_df.createOrReplaceTempView("home_sales_parquet")

In [73]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
temp_df = spark.sql("""
     Select view, date_built as year, concat('$', format_number(avg_price, 2)) AS avg_price
     from (
            select view, date_built, round(avg(price), 2) as avg_price
            from home_sales_parquet
            group by view, date_built
            having avg_price >= 350000
          ) as subquery
    order by view desc
""")
temp_df.show(5)

start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

+----+----+-------------+
|view|year|    avg_price|
+----+----+-------------+
| 100|2014|$1,048,368.75|
| 100|2015|  $716,333.50|
| 100|2017|  $805,038.00|
| 100|2012|$1,069,511.25|
| 100|2011|  $973,403.00|
+----+----+-------------+
only showing top 5 rows

--- 8.487701416015625e-05 seconds ---


In [74]:
# 14. Uncache the home_sales temporary table.
temp_df = spark.sql("UNCACHE TABLE home_sales")

In [75]:
# 15. Check if the home_sales is no longer cached
temp_df = spark.catalog.isCached('home_sales')